In [1]:
import pandas as pd
import numpy as np
import random
import datetime
import math

In [2]:
# generating data
def data_generator(train_num, test_num):
    training_set = []
    for i in range(train_num):
        y = 2*random.randint(0, 1) - 1  # 產生0則y=-1，產生1則y=1
        if y == 1:
            x1 = random.normalvariate(2, 0.6 ** 0.5)
            x2 = random.normalvariate(3, 0.6 ** 0.5)
        elif y == -1:
            x1 = random.normalvariate(0, 0.4 ** 0.5)
            x2 = random.normalvariate(4, 0.4 ** 0.5)
        training_set.append([1, x1, x2, y])

    testing_set = []
    for i in range(test_num):
        y = 2*random.randint(0, 1) - 1  # 產生0則y=-1，產生1則y=1
        if y == 1:
            x1 = random.normalvariate(2, 0.6 ** 0.5)
            x2 = random.normalvariate(3, 0.6 ** 0.5)
        elif y == -1:
            x1 = random.normalvariate(0, 0.4 ** 0.5)
            x2 = random.normalvariate(4, 0.4 ** 0.5)
        testing_set.append([1, x1, x2, y])
    train = pd.DataFrame(training_set, columns=['x0', 'x1', 'x2', 'y'])
    test = pd.DataFrame(testing_set, columns=['x0', 'x1', 'x2', 'y'])
    return train, test

In [3]:
def data_generator_with_outlier(train_num, test_num, outlier_num):
    training_set = []
    for i in range(train_num):
        y = 2*random.randint(0, 1) - 1  # 產生0則y=-1，產生1則y=1
        if y == 1:
            x1 = random.normalvariate(2, 0.6 ** 0.5)
            x2 = random.normalvariate(3, 0.6 ** 0.5)
        elif y == -1:
            x1 = random.normalvariate(0, 0.4 ** 0.5)
            x2 = random.normalvariate(4, 0.4 ** 0.5)
        training_set.append([1, x1, x2, y])
    
    for i in range(outlier_num):
        y = 1  # 全部label為1
        x1 = random.normalvariate(6, 0.3 ** 0.5)
        x2 = random.normalvariate(0, 0.1 ** 0.5)
        training_set.append([1, x1, x2, y])

    testing_set = []
    for i in range(test_num):
        y = 2*random.randint(0, 1) - 1  # 產生0則y=-1，產生1則y=1
        if y == 1:
            x1 = random.normalvariate(2, 0.6 ** 0.5)
            x2 = random.normalvariate(3, 0.6 ** 0.5)
        elif y == -1:
            x1 = random.normalvariate(0, 0.4 ** 0.5)
            x2 = random.normalvariate(4, 0.4 ** 0.5)
        testing_set.append([1, x1, x2, y])
    train = pd.DataFrame(training_set, columns=['x0', 'x1', 'x2', 'y'])
    test = pd.DataFrame(testing_set, columns=['x0', 'x1', 'x2', 'y'])
    return train, test

In [19]:
train_data, test_data = data_generator(train_num=200, test_num=5000)
train_data.head(3)

,x0,x1,x2,y
0,1,1.116427,3.624175,1
1,1,2.333818,3.244168,1
2,1,0.705970,2.505672,1


In [61]:
# Q13
total_ein = 0
for i in range(100):
    train_data, test_data = data_generator(train_num=200, test_num=5000)
    train_X = np.matrix(train_data.drop('y', axis=1))
    train_y = np.array(train_data['y'])
    w = np.linalg.inv(np.transpose(train_X).dot(train_X)).dot(np.transpose(train_X)).dot(train_y)
    
    ein = 0
    for j in range(len(train_X)):  # 跑每筆資料
        y_predict = w.dot(np.transpose(train_X[j])).item()
        ein += (y_predict - train_y[j]) ** 2
    ein /= len(train_X)
    total_ein += ein
total_ein /= 100
print(total_ein)

0.2811757551388505


In [63]:
# Q14
total_err_diff = 0
for i in range(100):
    train_data, test_data = data_generator(train_num=200, test_num=5000)
    train_X = np.matrix(train_data.drop('y', axis=1))
    train_y = np.array(train_data['y'])
    test_X = np.matrix(test_data.drop('y', axis=1))
    test_y = np.array(test_data['y'])
    w = np.linalg.inv(np.transpose(train_X).dot(train_X)).dot(np.transpose(train_X)).dot(train_y)
    
    # ein
    errin = 0
    for j in range(len(train_X)):  # 跑每筆資料
        y_predict = w.dot(np.transpose(train_X[j])).item()
        if y_predict >= 0:
            y_predict = 1
        else:
            y_predict = -1
        
        if y_predict != train_y[j]:
            errin += 1
    
    errin /= len(train_X)
    
    # eout
    errout = 0
    for j in range(len(test_X)):  # 跑每筆資料
        y_predict = w.dot(np.transpose(test_X[j])).item()
        if y_predict >= 0:
            y_predict = 1
        else:
            y_predict = -1
        
        if y_predict != test_y[j]:
            errout += 1
    
    errout /= len(test_X)
    
    total_err_diff += abs(errin - errout)
total_err_diff /= 100
print(total_err_diff)

0.013822


In [65]:
wlog = np.array([0.] * 3)
wlog

array([0., 0., 0.])

In [97]:
# Q15
total_eoutlr = 0
total_eoutlog = 0
for i in range(100):
    train_data, test_data = data_generator(train_num=200, test_num=5000)
    train_X = np.matrix(train_data.drop('y', axis=1))
    train_y = np.array(train_data['y'])
    test_X = np.matrix(test_data.drop('y', axis=1))
    test_y = np.array(test_data['y'])
    wlin = np.linalg.inv(np.transpose(train_X).dot(train_X)).dot(np.transpose(train_X)).dot(train_y)
    
    # eout of LR
    eoutlr = 0
    for j in range(len(test_X)):  # 跑每筆資料
        y_predict = wlin.dot(np.transpose(test_X[j])).item()
        if y_predict >= 0:
            y_predict = 1
        else:
            y_predict = -1
        
        if y_predict != test_y[j]:
            eoutlr += 1
    
    eoutlr /= len(test_X)
    total_eoutlr += eoutlr
    
    # training wlog
    learning_rate = 0.1
    wlog = np.array([0.] * 3)
    for t in range(500):
        for j in range(len(train_X)):
            s = -train_y[j] * wlog.dot(np.transpose(train_X[j])).item()
            if j == 0:
                gradient_einw = (1 / (1+math.exp(-s))) * (-train_y[j] * train_X[j])
            else:
                gradient_einw += (1 / (1+math.exp(-s))) * (-train_y[j] * train_X[j])
        gradient_einw /= len(train_X)
        gradient_einw = np.ravel(gradient_einw)
        wlog -= learning_rate * gradient_einw  # updating wlog
    # eout of Log
    eoutlog = 0
    for j in range(len(test_X)):  # 跑每筆資料
        s = wlog.dot(np.transpose(test_X[j])).item()
        prob = 1 / (1+math.exp(-s))
        
        if prob >= 0.5:
            y_predict = 1
        else:
            y_predict = -1
        
        if y_predict != test_y[j]:
            eoutlog += 1
    eoutlog /= len(test_X)
    total_eoutlog += eoutlog

total_eoutlr /= 100
total_eoutlog /= 100
print('Eout(A):', total_eoutlr)
print('Eout(B):', total_eoutlog)

Eout(A): 0.05807599999999998
Eout(B): 0.059755999999999976


In [4]:
# Q16
total_eoutlr = 0
total_eoutlog = 0
for i in range(100):
    train_data, test_data = data_generator_with_outlier(train_num=200, test_num=5000, outlier_num=20)
    
    train_X = np.matrix(train_data.drop('y', axis=1))
    train_y = np.array(train_data['y'])
    test_X = np.matrix(test_data.drop('y', axis=1))
    test_y = np.array(test_data['y'])
    wlin = np.linalg.inv(np.transpose(train_X).dot(train_X)).dot(np.transpose(train_X)).dot(train_y)
    
    # eout of LR
    eoutlr = 0
    for j in range(len(test_X)):  # 跑每筆資料
        y_predict = wlin.dot(np.transpose(test_X[j])).item()
        if y_predict >= 0:
            y_predict = 1
        else:
            y_predict = -1
        
        if y_predict != test_y[j]:
            eoutlr += 1
    
    eoutlr /= len(test_X)
    total_eoutlr += eoutlr
    
    # training wlog
    learning_rate = 0.1
    wlog = np.array([0.] * 3)
    for t in range(500):
        for j in range(len(train_X)):
            s = -train_y[j] * wlog.dot(np.transpose(train_X[j])).item()
            if j == 0:
                gradient_einw = (1 / (1+math.exp(-s))) * (-train_y[j] * train_X[j])
            else:
                gradient_einw += (1 / (1+math.exp(-s))) * (-train_y[j] * train_X[j])
        gradient_einw /= len(train_X)
        gradient_einw = np.ravel(gradient_einw)
        wlog -= learning_rate * gradient_einw  # updating wlog
    # eout of Log
    eoutlog = 0
    for j in range(len(test_X)):  # 跑每筆資料
        s = wlog.dot(np.transpose(test_X[j])).item()
        prob = 1 / (1+math.exp(-s))
        
        if prob >= 0.5:
            y_predict = 1
        else:
            y_predict = -1
        
        if y_predict != test_y[j]:
            eoutlog += 1
    eoutlog /= len(test_X)
    total_eoutlog += eoutlog

total_eoutlr /= 100
total_eoutlog /= 100
print('Eout(A):', total_eoutlr)
print('Eout(B):', total_eoutlog)

Eout(A): 0.09187800000000002
Eout(B): 0.05915800000000001
